<a href="https://colab.research.google.com/github/PaulSerin/Big-Data-Framework/blob/main/BDF_11_Graph_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#00 - Configuration of Apache Spark on Collaboratory


###Installing Java, Spark, and Findspark


---


This code installs Apache Spark 3.0.1, Java 8, and [Findspark](https://github.com/minrk/findspark), a library that makes it easy for Python to find Spark.

In [1]:
import os

os.environ["SPARK_VERSION"] = "spark-3.2.3"
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget  https://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!echo $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,172 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,619 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-s

In [2]:
!wget https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar

--2024-11-26 15:20:20--  https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar
Resolving repos.spark-packages.org (repos.spark-packages.org)... 3.168.132.68, 3.168.132.8, 3.168.132.114, ...
Connecting to repos.spark-packages.org (repos.spark-packages.org)|3.168.132.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 247880 (242K) [binary/octet-stream]
Saving to: ‘graphframes-0.8.2-spark3.2-s_2.12.jar’

graphframes-0.8.2-s 100%[===================>] 242.07K  --.-KB/s    in 0.03s   

2024-11-26 15:20:21 (8.12 MB/s) - ‘graphframes-0.8.2-spark3.2-s_2.12.jar’ saved [247880/247880]



### Set Environment Variables
Set the locations where Spark and Java are installed.

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark/"
os.environ["DRIVE_DATA"] = "/content/gdrive/My Drive/Big Data Framework/data/"

!rm /content/spark
!ln -s /content/$SPARK_VERSION-bin-hadoop2.7 /content/spark

!mv graphframes-0.8.2-spark3.2-s_2.12.jar /content/spark/jars/

!export SPARK_HOME=/content/spark
!export PATH=$PATH:$SPARK_HOME/bin:$SPARK_HOME/sbin

!echo $SPARK_HOME
!env |grep  "DRIVE_DATA"

!ls -l /content/

rm: cannot remove '/content/spark': No such file or directory
/content/spark/
DRIVE_DATA=/content/gdrive/My Drive/Big Data Framework/data/
total 266484
drwxr-xr-x  1 root root      4096 Nov 22 14:23 sample_data
lrwxrwxrwx  1 root root        34 Nov 26 15:20 spark -> /content/spark-3.2.3-bin-hadoop2.7
drwxr-xr-x 13  501 1000      4096 Nov 14  2022 spark-3.2.3-bin-hadoop2.7
-rw-r--r--  1 root root 272866820 Nov 14  2022 spark-3.2.3-bin-hadoop2.7.tgz


### Start a SparkSession
This will start a local Spark session.

In [4]:
!python -V

import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext.getOrCreate()

sc.addPyFile('/content/spark/jars/graphframes-0.8.2-spark3.2-s_2.12.jar')

# Example: shows the PySpark version
print("PySpark version {0}".format(sc.version))

# Example: parallelise an array and show the 2 first elements
sc.parallelize([2, 3, 4, 5, 6]).cache().take(2)

Python 3.10.12
PySpark version 3.2.3


[2, 3]

In [5]:
from pyspark.sql import SparkSession
# We create a SparkSession object (or we retrieve it if it is already created)
spark = SparkSession \
.builder \
.appName("My application") \
.config("packages","graphframes:graphframes-0.8.2-spark3.2-s_2.12") \
.master("local[4]") \
.getOrCreate()
# We get the SparkContext
sc = spark.sparkContext

In [6]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive



---


# 11 - Graph processing

## GraphX: Graph processing with RDDs

Parallel graph programming using Spark

- Main abstraction: [*Graph*](http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.graphx.Graph)
    -   Directed multigraph with properties assigned to vertices and edges
    -   It is an extension of the RDDs
- It includes graph constructors, basic operators ( *reverse*, *subgraph*…) and graph algorithms ( *PageRank*, *Triangle Counting*…)
- Only availabe on Scala.

Documentation: [spark.apache.org/docs/latest/graphx-programming-guide.html](http://spark.apache.org/docs/latest/graphx-programming-guide.html)

API: https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.graphx.package

## Graphs in GraphX
<img src="http://persoal.citius.usc.es/tf.pena/TCDM/figs/grapxgraph.png" alt="Grafo en GraphX" style="width: 50px;"/>
(Source: M.S. Malak, R. East "Spark GraphX in action", Manning, 2016)

### Example of a simple graph
<img src="http://persoal.citius.usc.es/tf.pena/TCDM/figs/simpsonsgraph.png" alt="Grafo de los Simpson" style="width: 600px;"/>
(Source: P. Zecević, M. Bonaći "Spark in action", Manning, 2017)

## GraphFrames: : Graph processing with DataFrames

In Python we can use [*GraphFrames*](https://graphframes.github.io/graphframes/docs/_site/quick-start.html) which wraps GraphX algorithms under the DataFrames API, providing a Python interface.

- Support for multiple languages is on the works
    - For now,  available for Scala and Python
- Not yet integrated on Spark
    - Available as an external package (https://spark-packages.org/package/graphframes/graphframes)

More information:
- Project web: https://graphframes.github.io/graphframes/docs/_site/
- Python API : https://graphframes.github.io/graphframes/docs/_site/api/python/index.html


### Graphs using pyspark and GraphFrames

In [7]:
# The following example shows how to create a GraphFrame, query it, and run the PageRank algorithm.
# Source: https://graphframes.github.io/graphframes/docs/_site/quick-start.html

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

from graphframes import *

# Create a Vertex DataFrame with unique ID column "id"
v = sqlContext.createDataFrame([
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 30),
], ["id", "name", "age"])
# Create an Edge DataFrame with "src" and "dst" columns
e = sqlContext.createDataFrame([
  ("a", "b", "friend"),
  ("b", "c", "follow"),
  ("c", "b", "follow"),
], ["src", "dst", "relationship"])
# Create a GraphFrame

g = GraphFrame(v, e)

# Query: Get in-degree of each vertex.
g.inDegrees.show()

# Query: Count the number of "follow" connections in the graph.
g.edges.filter("relationship = 'follow'").count()

# Run PageRank algorithm, and show results.
results = g.pageRank(resetProbability=0.01, maxIter=20)
results.vertices.select("id", "pagerank").show()

/content/spark/python/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+---+--------+
| id|inDegree|
+---+--------+
|  b|       2|
|  c|       1|
+---+--------+

+---+------------------+
| id|          pagerank|
+---+------------------+
|  c|1.8994109890559092|
|  b|1.0905890109440908|
|  a|              0.01|
+---+------------------+



#Exercises

## Exercise 11.1:

A long time ago in a galaxy far, far away, the characters of the Star Wars franchise interacted with each other in an endless series of films. An ancient Jedi order, called the *Data Guardians of the Galaxy* (not affiliated to Marvel's homonym :) registered all those interactions and saved them on a digital file so that they could be studied by the forthcoming generations. This file was originally called (guess it) `sw.txt`, and you will find it in the `/data` directory.

Using pySpark, perform the following operations and answer the following questions:

1. Load the `$DRIVE_DATA/sw.txt` file. Take into account that it is a JSON file.
2. Using this information, create a graph of interactions between the Star Wars characters.
3. How many different characters are there?
4. How many interactions are there?
5. Who is the central character in Star Wars (the one who interacts in most scenes)?
6. Who is the character with the highest 'rank' in Star Wars (use the PageRank algorithm)?

In [15]:
from re import MULTILINE
sw_DF = spark.read.json(os.environ["DRIVE_DATA"] + "sw.txt", multiLine=True).cache()
sw_DF.printSchema()
# Show the first few rows
sw_DF.show(truncate=False)

root
 |-- links: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- source: long (nullable = true)
 |    |    |-- target: long (nullable = true)
 |    |    |-- value: long (nullable = true)
 |-- nodes: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- colour: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- value: long (nullable = true)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [16]:
# Explode the 'nodes' array to get individual characters
characters_DF = sw_DF.selectExpr("explode(nodes) as node") \
                     .selectExpr("node.name as name", "node.value as interactions", "node.colour as colour")

# Show the extracted data
characters_DF.show(truncate=False)


+--------------+------------+-------+
|name          |interactions|colour |
+--------------+------------+-------+
|DARTH VADER   |190         |#000000|
|R2-D2         |171         |#bde0f6|
|CHEWBACCA     |145         |#A0522D|
|BB-8          |40          |#eb5d00|
|QUI-GON       |62          |#4f4fb1|
|NUTE GUNRAY   |25          |#808080|
|PK-4          |4           |#808080|
|TC-14         |5           |#808080|
|OBI-WAN       |148         |#48D1CC|
|DOFINE        |4           |#808080|
|RUNE          |11          |#808080|
|TEY HOW       |5           |#808080|
|EMPEROR       |52          |#191970|
|CAPTAIN PANAKA|20          |#808080|
|SIO BIBBLE    |9           |#808080|
|JAR JAR       |42          |#9a9a00|
|TARPALS       |4           |#808080|
|BOSS NASS     |5           |#808080|
|PADME         |75          |#DDA0DD|
|RIC OLIE      |12          |#808080|
+--------------+------------+-------+
only showing top 20 rows



In [17]:
# Count the number of distinct characters
num_characters = characters_DF.select("name").distinct().count()
print(f"Number of different characters: {num_characters}")

Number of different characters: 111


In [21]:
# Find the character with the maximum interactions
central_character = characters_DF.orderBy("interactions", ascending=False).first()
print(f"Central character: {central_character['name']} with {central_character['interactions']} interactions")

Central character: DARTH VADER with 190 interactions


In [22]:
from pyspark.sql.functions import col, explode, lit

# Create a DataFrame for edges (pairwise connections between all characters)
edges_DF = characters_DF.crossJoin(characters_DF.withColumnRenamed("name", "name_1")) \
                        .filter(col("name") != col("name_1")) \
                        .select(col("name").alias("src"), col("name_1").alias("dst"))

# Show the edges
edges_DF.show(truncate=False)

+-----------+--------------+
|src        |dst           |
+-----------+--------------+
|DARTH VADER|R2-D2         |
|DARTH VADER|CHEWBACCA     |
|DARTH VADER|BB-8          |
|DARTH VADER|QUI-GON       |
|DARTH VADER|NUTE GUNRAY   |
|DARTH VADER|PK-4          |
|DARTH VADER|TC-14         |
|DARTH VADER|OBI-WAN       |
|DARTH VADER|DOFINE        |
|DARTH VADER|RUNE          |
|DARTH VADER|TEY HOW       |
|DARTH VADER|EMPEROR       |
|DARTH VADER|CAPTAIN PANAKA|
|DARTH VADER|SIO BIBBLE    |
|DARTH VADER|JAR JAR       |
|DARTH VADER|TARPALS       |
|DARTH VADER|BOSS NASS     |
|DARTH VADER|PADME         |
|DARTH VADER|RIC OLIE      |
|DARTH VADER|WATTO         |
+-----------+--------------+
only showing top 20 rows



In [23]:
from graphframes import GraphFrame

# Create a GraphFrame
vertices = characters_DF.select(col("name").alias("id"))
graph = GraphFrame(vertices, edges_DF)

# Run PageRank
page_rank = graph.pageRank(resetProbability=0.15, maxIter=10)

# Show PageRank results
page_rank.vertices.select("id", "pagerank").orderBy("pagerank", ascending=False).show(truncate=False)

# Find the character with the highest rank
highest_rank = page_rank.vertices.orderBy("pagerank", ascending=False).first()
print(f"Character with highest PageRank: {highest_rank['id']} with rank {highest_rank['pagerank']}")


+--------------+------------------+
|id            |pagerank          |
+--------------+------------------+
|DARTH VADER   |0.9999999999999987|
|R2-D2         |0.9999999999999987|
|CHEWBACCA     |0.9999999999999987|
|BB-8          |0.9999999999999987|
|QUI-GON       |0.9999999999999987|
|NUTE GUNRAY   |0.9999999999999987|
|PK-4          |0.9999999999999987|
|TC-14         |0.9999999999999987|
|OBI-WAN       |0.9999999999999987|
|DOFINE        |0.9999999999999987|
|RUNE          |0.9999999999999987|
|TEY HOW       |0.9999999999999987|
|EMPEROR       |0.9999999999999987|
|CAPTAIN PANAKA|0.9999999999999987|
|SIO BIBBLE    |0.9999999999999987|
|JAR JAR       |0.9999999999999987|
|TARPALS       |0.9999999999999987|
|BOSS NASS     |0.9999999999999987|
|PADME         |0.9999999999999987|
|RIC OLIE      |0.9999999999999987|
+--------------+------------------+
only showing top 20 rows

Character with highest PageRank: DARTH VADER with rank 0.9999999999999987
